### Load PyTorch Model

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

# Import the s4 model path
import sys
sys.path.append('/Users/poomchan/Developer/light-har/code/s4')
from s4d import S4D

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
torch.__version__

'1.13.1'

In [4]:
class S4Model(nn.Module):
    def __init__(
        self,
        d_input,
        d_output,
        d_model=256,
        n_layers=4,
        dropout=0.2,
        lr=0.001,
        dropout_fn=nn.Dropout,
        prenorm=False,
    ):
        super().__init__()

        self.prenorm = prenorm

        # Linear encoder (d_input = 1 for grayscale and 3 for RGB)
        self.encoder = nn.Linear(d_input, d_model)

        # Stack S4 layers as residual blocks
        self.s4_layers = nn.ModuleList()
        self.norms = nn.ModuleList()
        self.dropouts = nn.ModuleList()
        for _ in range(n_layers):
            self.s4_layers.append(
                S4D(d_model, dropout=dropout, transposed=True, lr=lr)
            )
            self.norms.append(nn.LayerNorm(d_model))
            self.dropouts.append(dropout_fn(dropout))

        # Linear decoder
        self.decoder = nn.Linear(d_model, d_output)

    def forward(self, x):
        """
        Input x is shape (B, L, d_input)
        """
        x = self.encoder(x)  # (B, L, d_input) -> (B, L, d_model)

        x = x.transpose(-1, -2)  # (B, L, d_model) -> (B, d_model, L)
        for layer, norm, dropout in zip(self.s4_layers, self.norms, self.dropouts):
            # Each iteration of this loop will map (B, d_model, L) -> (B, d_model, L)

            z = x
            if self.prenorm:
                # Prenorm
                z = norm(z.transpose(-1, -2)).transpose(-1, -2)

            # Apply S4 block: we ignore the state input and output
            z, _ = layer(z)

            # Dropout on the output of the S4 block
            z = dropout(z)

            # Residual connection
            x = z + x

            if not self.prenorm:
                # Postnorm
                x = norm(x.transpose(-1, -2)).transpose(-1, -2)

        x = x.transpose(-1, -2)

        # Pooling: average pooling over the sequence length
        x = x.mean(dim=1)

        # Decode the outputs
        x = self.decoder(x)  # (B, d_model) -> (B, d_output)

        return x


In [5]:
# import the PyTorch model.
model = S4Model(
    d_input=3, # num of feature
    d_output=6, # 6 classes
    d_model=16,
    n_layers=4,
    dropout=0.2,
    lr=0.001,
    dropout_fn=nn.Dropout,
    prenorm=False,
)

model.eval()
model_path = "/Users/poomchan/Developer/light-har/code/s4/models/s4-d16.pt"
model.load_state_dict(torch.load(model_path, map_location='cpu'))

<All keys matched successfully>

### Convert to CoreML

In [6]:
import coremltools as ct

example_input = torch.randn(1, 16, 3)

model.to('cpu')
traced_model = torch.jit.trace(model, example_input)
traced_model(example_input)

# Convert to Core ML program using the Unified Conversion API.
coreml_model = ct.convert(
    traced_model,
    convert_to="mlprogram",
    inputs=[ct.TensorType(shape=example_input.shape)]
 )

# Save the converted model.
coreml_model.save("s4.mlpackage")

/Users/poomchan/miniconda3/envs/coreml/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/poomchan/miniconda3/envs/coreml/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.8.dylib
  Referenced from: <90B45554-B79A-39BA-8A44-F2AD877B729C> /Users/poomchan/miniconda3/envs/coreml/lib/python3.9/site-packages/torchvision/image.so
  Reason: tried: '/Users/poomchan/miniconda3/envs/coreml/lib/python3.9/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/poomchan/miniconda3/envs/coreml/lib/python3.9/site-packages/torchvision/../../../libjpeg.8.dylib' (no such file), '/Users/poomchan/miniconda3/envs/coreml/lib/python3.9/lib-dynload/../../libjpeg.8.dylib' (no such file), '/Users/poomchan/miniconda3/envs/coreml/bin/../lib/libjpeg.8.dylib' (no such file), '/usr/local/lib/libjpeg.8.dylib' (no such file), '/usr/lib/libjpeg.8.dylib' (no such file, not in dyld cache)

AttributeError: 'torch._C.Node' object has no attribute 'cs'

### Perform Quantization

In [ ]:
import coremltools.optimize.coreml as cto
import coremltools as ct

coreml_model = ct.models.MLModel("cnn.mlpackage")
coreml_model

In [ ]:
op_config = cto.OpLinearQuantizerConfig(mode="linear_symmetric", weight_threshold=512)
config = cto.OptimizationConfig(global_config=op_config)

quantized_model = cto.linear_quantize_weights(coreml_model, config=config)

# Save the model
model_path = "quantized-s4.mlpackage"
quantized_model.save("quantized-s4.mlpackage")

### Test the model

In [ ]:
from numpy import genfromtxt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load Data
x = genfromtxt('../Data/WISDM_x.csv', delimiter=',')
y_df = pd.read_csv('../Data/WISDM_y.csv')
y = y_df.values.flatten()  # Flatten if y is 2D

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Function to create time series dataset
def create_series(x, y, timestep, overlap):
    slide_step = int(timestep * (1 - overlap))
    data_num = int((len(x) / slide_step) - 1)
    dataset = np.ndarray(shape=(data_num, timestep, x.shape[1]))
    labels = []

    for i in range(data_num):
        labels.append(y[slide_step * (i + 1) - 1])
        for j in range(timestep):
            dataset[i, j, :] = x[slide_step * i + j, :]

    return dataset, np.array(labels)

# Create time series
timestep = 16  # Replace with your value
overlap = 0.5  # Replace with your value
X_series, y_series = create_series(x, y_encoded, timestep, overlap)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_series, y_series, test_size=0.2, random_state=42)
print(f'X_train shape:{X_train.shape}, X_test shape:{X_test.shape}, y_train shape:{y_train.shape}, y_test shape:{y_test.shape}')

# Convert arrays to PyTorch Tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)  # Assuming y_train is class labels for classification
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Creating TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Creating DataLoaders
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
from pathlib import Path

def coreml_metrics(model_name, X_test, y_test, model_path):
    predictions = []
    for i in range(len(X_test)):
        X_test_sample = X_test[i].view(1, 3, 16)
        #X_test_new = np.expand_dims(X_test[id], axis=0)
        output_dict = model_name.predict({'x': X_test_sample.numpy()})
        pred_class = np.argmax(output_dict['var_75'])
        predictions.append(pred_class)
    
    accuracy = np.sum(np.array(predictions) == y_test.numpy()) / len(predictions)
    print("Accuracy:", accuracy)
    
    model_file = Path(model_path)
    
    # Size in bytes
    model_size_bytes = model_file.stat().st_size
    
    # Convert size to kilobytes (optional)
    model_size_kb = model_size_bytes / 1024
    print(f"Size of the model: {model_size_kb:.2f} KB")

In [ ]:
coreml_metrics(quantized_model, X_test, y_test, model_path)